Author: Lester Hedges and Christopher Woods<br>
Email:&nbsp;&nbsp; lester.hedges@bristol.ac.uk, christopher.woods@bristol.ac.uk

# Molecular setup

In this notebook you'll learn how to use BioSimSpace to parameterise and solvate molecules.

First we'll need to import BioSimSpace:

In [ ]:
import BioSimSpace as BSS

First we will load in benzene and parameterise it using GAFF2

In [ ]:
molecule = BSS.IO.readMolecules("molecules/benzene.pdb")[0]

Now let's paramterise the molecule. We do so by calling the `parameterise` function from the `BSS.Parameters` package, passing the molecule and force field name as arguments. Since parameterisation can be slow, the function returns a handle to a process that runs the parameterisation in the background. To get the parameterised molecule from the process we need to call the `getMolecule` method. This is a blocking operation which waits for the process to finish before grabbing the molecule and returning it. 

In [ ]:
molecule = BSS.Parameters.parameterise(molecule, "gaff2").getMolecule()

We can see that it is parameterised by printing out all of the atoms. Note how symmetrical atoms have symmetrical charges :-)

In [ ]:
for atom in molecule._getSireObject().atoms():
    print(atom, atom.property("charge"), atom.property("LJ"))

Next we will solvate our molecule in a box of water using the `solvate` function from the `BSS.Solvent` package. This will centre the molecule in a cubic box of a specified size and surround it by water molecules. Here we allow the user to specify the size of the box and the ionic strength (by default, the resulting system will also be neutralised too).

Note that the molecule is an optional `keyword` argument to the solvate function. This is because its also possible to create a pure water box, i.e. without any molecules in it.

In [ ]:
system = BSS.Solvent.solvate("tip3p", molecule=molecule,
                                      box=3*[4*BSS.Units.Length.nanometer])

We now write the output as GROMACS format files representing the parameterised and solvated molecular system.

In [ ]:
BSS.IO.saveMolecules("solvated_benzene", system, ["grotop", "gro87"])

In [ ]:
!cat solvated_benzene.grotop

Finally, let's now take a look at the solvated structure :-)

In [ ]:
BSS.viewMolecules(["solvated_benzene.grotop","solvated_benzene.gro87"])

Ok - that was a small molecule. How about something bigger? Let's now load up a protein...

In [ ]:
molecule = BSS.IO.readMolecules("molecules/2JJC.pdb")[0]

Now we will parameterise this using FF14SB...

In [ ]:
molecule = BSS.Parameters.parameterise(molecule, "ff14SB").getMolecule()

In [ ]:
for atom in molecule._getSireObject().atoms():
    print(atom, atom.property("charge"),atom.property("LJ"))

Now let's solvate this in a box of TIP4P...

In [ ]:
system = BSS.Solvent.solvate("tip3p", molecule=molecule,
                                      box=3*[5*BSS.Units.Length.nanometer])

Now we will save this to a Amber PRM7, NETCDF and PDB formats...

In [ ]:
BSS.IO.saveMolecules("solvated_protein", system, ["prm7", "rst", "pdb"])

In [ ]:
!head -500 solvated_protein.prm7

What does this look like?

In [ ]:
BSS.viewMolecules("solvated_protein.pdb")